In [ ]:
%load_ext autoreload
%autoreload 2

from fastai.imports import *

#from fastai.structured import *

from pandas_summary import DataFrameSummary

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from IPython.display import display

from sklearn import metrics

In [ ]:
from sklearn.model_selection import *

In [ ]:
from feather import *

In [ ]:
PATH = ""

In [ ]:
df_raw = pd.read_csv('../input/train.csv', low_memory=False)

In [ ]:
df_test = pd.read_csv('../input/test.csv', low_memory = False)

In [ ]:
df_raw.head()

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows",1000, "display.max_columns",1000):
        display(df)

In [ ]:
display_all(df_raw.head().T)

In [ ]:
df_train, df_valid, y_train, y_valid = train_test_split(df_raw.drop(['Id','Cover_Type'], axis = 1), df_raw.Cover_Type, test_size=0.20, random_state=42)

In [ ]:
df_train.shape

In [ ]:
df_valid.shape

### training a model

In [ ]:
m = RandomForestClassifier(n_jobs = -1)

In [ ]:
m.fit(df_train, y_train)

In [ ]:
def print_score(m):
    res = [m.score(df_train, y_train), m.score(df_valid, y_valid)]
    if hasattr(m, 'oob_score_'):res.append(m.oob_score_)
    print (res)

In [ ]:
m = RandomForestClassifier(n_jobs = -1)
%time m.fit(df_train, y_train)
print_score(m)

### let's try a single tree?

In [ ]:
m = RandomForestClassifier(n_estimators = 1, max_depth = 3, bootstrap = False, n_jobs = -1)
%time m.fit(df_train, y_train)
print_score(m)

In [ ]:
m = RandomForestClassifier(n_estimators = 1, bootstrap = False, n_jobs = -1)
%time m.fit(df_train, y_train)
print_score(m)

### multiple trees with bootstrapping

In [ ]:
m = RandomForestClassifier (n_jobs = -1)
%time m.fit(df_train, y_train)
print_score(m)

In [ ]:
preds = np.stack([t.predict(df_valid) for t in m.estimators_])

In [ ]:
preds[:,:3]

In [ ]:
m = RandomForestClassifier (n_jobs = -1, n_estimators = 20)
%time m.fit(df_train, y_train)
print_score(m)

In [ ]:
m = RandomForestClassifier (n_jobs = -1, n_estimators = 40)
%time m.fit(df_train, y_train)
print_score(m)

In [ ]:
m = RandomForestClassifier (n_jobs = -1, n_estimators = 160)
%time m.fit(df_train, y_train)
print_score(m)

In [ ]:
m = RandomForestClassifier (n_jobs = -1, n_estimators = 40, oob_score = True)
%time m.fit(df_train, y_train)
print_score(m)

### Subsample and Use the entire dataset

In [ ]:
m = RandomForestClassifier(n_jobs = -1, n_estimators = 40, oob_score = True)
%time m.fit(df_train, y_train)
print_score(m)

### excluding subsample and using all samples

In [ ]:
m = RandomForestClassifier(n_jobs = -1, n_estimators = 40, oob_score = True)
%time m.fit(df_train, y_train)
print_score(m)

In [ ]:
m = RandomForestClassifier(n_jobs = -1, n_estimators = 40, min_samples_leaf = 3, oob_score = True)
%time m.fit(df_train, y_train)
print_score(m)

In [ ]:
m = RandomForestClassifier(n_jobs = -1, n_estimators = 40, min_samples_leaf = 2, oob_score = True, max_features = 0.5)
%time m.fit(df_train, y_train)
print_score(m)

### predicting on the test dataset

In [ ]:
df_valid.head()

In [ ]:
df_test['Cover_Type'] = m.predict(df_test.drop('Id', axis = 1))

In [ ]:
df_test.Cover_Type.value_counts()

In [ ]:
df_test[['Id', 'Cover_Type']].to_csv('submission.csv', index=False)